# Table 7.SM.7

Theme Song: Vapour Trail<br>
Artist: Ride<br>
Album: Nowhere<br>
Released: 1990

Takes in gas properties and makes a nice formatted table, hopefully

In [1]:
import pandas as pd
import numpy as np
import copy
import urllib
from urllib.error import HTTPError
import time

pd.set_option('display.max_rows', None)

In [2]:
df_supp = pd.read_csv('../data_input/ghg_properties/metrics_supplement.csv')

In [3]:
#df_supp.index.name = "Name"

In [4]:
# strip whitespace and hashes
df = df_supp.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df = df.apply(lambda x: x.str.strip('#') if x.dtype == "object" else x)

In [5]:
# replace "nan" with np.nan
df.replace(to_replace="nan", value=np.nan, inplace=True)

In [6]:
df

,# Name,CASRN,Acronym,Formula,Lifetime,Radiative_efficiency,AGWP20,GWP20,AGWP100,GWP100,AGWP500,GWP500,AGTP50,GTP50,AGTP100,GTP100,CGTP50,CGTP100
0,Carbon dioxide,,,CO2,NaN,0.000013,2.430000e-14,1.000,8.950000e-14,1.000,3.140000e-13,1.000,4.280000e-16,1.000,3.950000e-16,1.000,0,0
1,Methane,,,CH4,11.800,0.000388,1.980000e-12,81.199,2.490000e-12,27.859,2.500000e-12,7.953,4.730000e-15,11.049,2.120000e-15,5.377,2732.019,3323.34
2,Nitrous oxide,,,N2O,109.000,0.003196,6.650000e-12,273.255,2.450000e-11,273.351,4.070000e-11,129.715,1.240000e-13,290.303,9.190000e-14,232.964,0,0
3,Trichlorofluoromethane,75-69-4,CFC-11,CCl3F,52.000,0.259000,1.810000e-10,7429.247,4.970000e-10,5559.066,5.860000e-10,1868.702,2.430000e-12,5670.234,1.250000e-12,3156.699,NaN,NaN
4,Dichlorodifluoromethane,75-71-8,CFC-12,CCl2F2,102.000,0.320000,2.770000e-10,11381.298,9.980000e-10,11156.036,1.600000e-09,5100.583,5.060000e-12,11840.660,3.660000e-12,9267.045,NaN,NaN
5,Chlorotrifluoromethane,75-72-9,CFC-13,CClF3,640.000,0.278000,3.010000e-10,12381.380,1.450000e-09,16227.164,5.500000e-09,17526.310,7.260000e-12,16980.901,7.400000e-12,18756.083,NaN,NaN
6,1:2-Difluoro-1:1:2:2-tetrachlo,76-12-0,CFC-112,CCl2FCCl2F,63.600,0.282000,1.370000e-10,5622.796,4.130000e-10,4617.974,5.250000e-10,1673.683,2.060000e-12,4811.292,1.190000e-12,3015.489,NaN,NaN
7,2:2-Difluoro-1:2:2:2-tetrachlo,76-11-9,CFC-112a,CCl3CClF2,52.000,0.246000,1.150000e-10,4741.008,3.170000e-10,3547.543,3.740000e-10,1192.521,1.550000e-12,3618.486,7.950000e-13,2014.462,NaN,NaN
8,1:1:2-Trichloro-1:2:2-trifluor,76-13-1,CFC-113,CCl2FCClF2,93.000,0.301000,1.670000e-10,6855.657,5.830000e-10,6520.812,8.900000e-10,2834.865,2.960000e-12,6911.091,2.060000e-12,5211.157,NaN,NaN
9,1:1:1-Trichloro-2:2:2-trifluor,354-58-5,CFC-113a,CCl3CF3,55.000,0.241000,1.240000e-10,5108.246,3.510000e-10,3926.850,4.220000e-10,1345.800,1.730000e-12,4033.876,9.170000e-13,2324.063,NaN,NaN


In [7]:
# resolve typos and omissions in CASRN
df.loc[71, 'CASRN'] = '29118-24-9'
df.loc[73, 'CASRN'] = '66711-86-2'
df.loc[182, 'CASRN'] = '163702-05-4'

# consistency with acronyms
df.loc[76, 'Acronym'] = 'HFO-1345zfc'
df.loc[182, 'Acronym'] = 'HFE-569sf2'
df.loc[107, 'Acronym'] = 'Halon-2311'
df.loc[148, 'Acronym'] = 'HCFE-235ca2'
df.loc[149, 'Acronym'] = 'HCFE-235da2'
df.loc[150, 'Acronym'] = 'HCFE-236ea2'
df.loc[163, 'Acronym'] = 'HFE-347mmz1'
df.loc[164, 'Acronym'] = 'HFE-347mcc3'
df.loc[178, 'Acronym'] = 'HFE-43-10pccc124'
df.loc[179, 'Acronym'] = 'HFE-449s1'
df.loc[186, 'Acronym'] = 'HFE-236ca12'
df.loc[187, 'Acronym'] = 'HFE-338pcc13'
df.loc[202, 'Acronym'] = 'PFPMIE'
df.loc[218, 'Acronym'] = ''
df.loc[225, 'Acronym'] = ''

In [8]:
df.rename(
    columns = {
        '# Name' : 'Name',
        'Radiative_efficiency' : 'Radiative efficiency (W m-2 ppb-1)',
        'Lifetime' : 'Lifetime (yr)',
        'CASRN': 'CAS'
    },
    inplace=True
)
df

,Name,CAS,Acronym,Formula,Lifetime (yr),Radiative efficiency (W m-2 ppb-1),AGWP20,GWP20,AGWP100,GWP100,AGWP500,GWP500,AGTP50,GTP50,AGTP100,GTP100,CGTP50,CGTP100
0,Carbon dioxide,,,CO2,NaN,0.000013,2.430000e-14,1.000,8.950000e-14,1.000,3.140000e-13,1.000,4.280000e-16,1.000,3.950000e-16,1.000,0,0
1,Methane,,,CH4,11.800,0.000388,1.980000e-12,81.199,2.490000e-12,27.859,2.500000e-12,7.953,4.730000e-15,11.049,2.120000e-15,5.377,2732.019,3323.34
2,Nitrous oxide,,,N2O,109.000,0.003196,6.650000e-12,273.255,2.450000e-11,273.351,4.070000e-11,129.715,1.240000e-13,290.303,9.190000e-14,232.964,0,0
3,Trichlorofluoromethane,75-69-4,CFC-11,CCl3F,52.000,0.259000,1.810000e-10,7429.247,4.970000e-10,5559.066,5.860000e-10,1868.702,2.430000e-12,5670.234,1.250000e-12,3156.699,NaN,NaN
4,Dichlorodifluoromethane,75-71-8,CFC-12,CCl2F2,102.000,0.320000,2.770000e-10,11381.298,9.980000e-10,11156.036,1.600000e-09,5100.583,5.060000e-12,11840.660,3.660000e-12,9267.045,NaN,NaN
5,Chlorotrifluoromethane,75-72-9,CFC-13,CClF3,640.000,0.278000,3.010000e-10,12381.380,1.450000e-09,16227.164,5.500000e-09,17526.310,7.260000e-12,16980.901,7.400000e-12,18756.083,NaN,NaN
6,1:2-Difluoro-1:1:2:2-tetrachlo,76-12-0,CFC-112,CCl2FCCl2F,63.600,0.282000,1.370000e-10,5622.796,4.130000e-10,4617.974,5.250000e-10,1673.683,2.060000e-12,4811.292,1.190000e-12,3015.489,NaN,NaN
7,2:2-Difluoro-1:2:2:2-tetrachlo,76-11-9,CFC-112a,CCl3CClF2,52.000,0.246000,1.150000e-10,4741.008,3.170000e-10,3547.543,3.740000e-10,1192.521,1.550000e-12,3618.486,7.950000e-13,2014.462,NaN,NaN
8,1:1:2-Trichloro-1:2:2-trifluor,76-13-1,CFC-113,CCl2FCClF2,93.000,0.301000,1.670000e-10,6855.657,5.830000e-10,6520.812,8.900000e-10,2834.865,2.960000e-12,6911.091,2.060000e-12,5211.157,NaN,NaN
9,1:1:1-Trichloro-2:2:2-trifluor,354-58-5,CFC-113a,CCl3CF3,55.000,0.241000,1.240000e-10,5108.246,3.510000e-10,3926.850,4.220000e-10,1345.800,1.730000e-12,4033.876,9.170000e-13,2324.063,NaN,NaN


In [9]:
# get IUPAC names from API where they are truncated, ambigious, wrong, have colons instead of commas...
# easier to specifiy which compounds are correct

good_idx = [0,1,2,3,4,5,17,18,19,39,40,41,49,65,67,90,91,92,93,94,98,99,100,101,102,103,104,105,110,111,112,114,115,
            116,117,118,119,120,121,122,124,125,126,127,128,129,130,131,132,133,134,135,142,143,204,220,228,229,230,
            232,234,235,236,237,240,241,242,243,244,245,246,247,248]

#def first_cap(string):
#    for j, c in enumerate(string):
#        if c.alpha():
#            break
#    return string[:j] + string[j:].capitalize()
    
    
# #iupac_list = []
for i, cas in enumerate(df['CAS']):
    if cas == '':
        continue
    if i not in good_idx:
        url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/%s/property/IUPACName/TXT" % cas
        try:
            file = urllib.request.urlopen(url)
            for line in file:
                iupac_name = line.decode('utf-8').strip()
                break
            print(cas, iupac_name)
            df.loc[i, 'Name'] = iupac_name.capitalize()
        #    iupac_list.append(iupac_name)
        except HTTPError:
            print('\n*** no entry found for %s\n' % cas)
        time.sleep(0.25)  # web server requests no more than 5 requests per second
    else:
        df.loc[i, 'Name'] = df.loc[i, 'Name'].capitalize()

76-12-0 1,1,2,2-tetrachloro-1,2-difluoroethane
76-11-9 1,1,1,2-tetrachloro-2,2-difluoroethane
76-13-1 1,1,2-trichloro-1,2,2-trifluoroethane
354-58-5 1,1,1-trichloro-2,2,2-trifluoroethane
76-14-2 1,2-dichloro-1,1,2,2-tetrafluoroethane
374-07-2 1,1-dichloro-1,2,2,2-tetrafluoroethane
76-15-3 1-chloro-1,1,2,2,2-pentafluoroethane
3832-15-3 (1S,2S)-1,2-dichloro-1,2,3,3,4,4-hexafluorocyclobutane
3934-26-7 (1S,2R)-1,2-dichloro-1,2,3,3,4,4-hexafluorocyclobutane
598-88-9 (E)-1,2-dichloro-1,2-difluoroethene
79-35-6 1,1-dichloro-2,2-difluoroethene
354-14-3 1,1,2,2-tetrachloro-1-fluoroethane
354-21-2 1,2,2-trichloro-1,1-difluoroethane
354-15-4 1,1,2-trichloro-1,2-difluoroethane
306-83-2 2,2-dichloro-1,1,1-trifluoroethane
354-23-4 1,2-dichloro-1,1,2-trifluoroethane
2837-89-0 2-chloro-1,1,1,2-tetrafluoroethane
354-25-6 1-chloro-1,1,2,2-tetrafluoroethane
431-06-1 1,2-dichloro-1,2-difluoroethane
471-43-2 1,1-dichloro-2,2-difluoroethane
1842-05-3 1,1-dichloro-1,2-difluoroethane
75-88-7 2-chloro-1,1,1-tr

928-95-0 (E)-hex-2-en-1-ol


In [10]:
# TODO: move to utility functions
def significance(x, digits):
    return (np.round(x, digits - (1 + int(np.floor(np.log10(np.abs(x)))))))

In [11]:
# all values to 3 SF
for col in ['Lifetime (yr)', 'Radiative efficiency (W m-2 ppb-1)', 'AGWP20', 'GWP20', 'AGWP100', 'GWP100', 'AGWP500', 'GWP500',
            'AGTP50', 'GTP50', 'AGTP100', 'GTP100', 'CGTP50', 'CGTP100']:
    df.loc[:,col] = pd.to_numeric(df.loc[:,col])
    df.loc[df[col] == 0,col] = 0
    df.loc[df[col] > 0,col] = df.loc[df[col] > 0,col].apply(significance, args=(3,))

In [12]:
# stop excel interpreting CAS numbers as dates
for i, row in df.iterrows():
    df.loc[i,'CAS'] = '="'+df.loc[i,'CAS']+'"'

In [13]:
df

,Name,CAS,Acronym,Formula,Lifetime (yr),Radiative efficiency (W m-2 ppb-1),AGWP20,GWP20,AGWP100,GWP100,AGWP500,GWP500,AGTP50,GTP50,AGTP100,GTP100,CGTP50,CGTP100
0,Carbon dioxide,"=""""",,CO2,NaN,0.000013,2.430000e-14,1.000,8.950000e-14,1.000,3.140000e-13,1.000,4.280000e-16,1.000,3.950000e-16,1.000,0.000,0.000
1,Methane,"=""""",,CH4,11.800,0.000388,1.980000e-12,81.200,2.490000e-12,27.900,2.500000e-12,7.950,4.730000e-15,11.000,2.120000e-15,5.380,2730.000,3320.000
2,Nitrous oxide,"=""""",,N2O,109.000,0.003200,6.650000e-12,273.000,2.450000e-11,273.000,4.070000e-11,130.000,1.240000e-13,290.000,9.190000e-14,233.000,0.000,0.000
3,Trichlorofluoromethane,"=""75-69-4""",CFC-11,CCl3F,52.000,0.259000,1.810000e-10,7430.000,4.970000e-10,5560.000,5.860000e-10,1870.000,2.430000e-12,5670.000,1.250000e-12,3160.000,NaN,NaN
4,Dichlorodifluoromethane,"=""75-71-8""",CFC-12,CCl2F2,102.000,0.320000,2.770000e-10,11400.000,9.980000e-10,11200.000,1.600000e-09,5100.000,5.060000e-12,11800.000,3.660000e-12,9270.000,NaN,NaN
5,Chlorotrifluoromethane,"=""75-72-9""",CFC-13,CClF3,640.000,0.278000,3.010000e-10,12400.000,1.450000e-09,16200.000,5.500000e-09,17500.000,7.260000e-12,17000.000,7.400000e-12,18800.000,NaN,NaN
6,"1,1,2,2-tetrachloro-1,2-difluoroethane","=""76-12-0""",CFC-112,CCl2FCCl2F,63.600,0.282000,1.370000e-10,5620.000,4.130000e-10,4620.000,5.250000e-10,1670.000,2.060000e-12,4810.000,1.190000e-12,3020.000,NaN,NaN
7,"1,1,1,2-tetrachloro-2,2-difluoroethane","=""76-11-9""",CFC-112a,CCl3CClF2,52.000,0.246000,1.150000e-10,4740.000,3.170000e-10,3550.000,3.740000e-10,1190.000,1.550000e-12,3620.000,7.950000e-13,2010.000,NaN,NaN
8,"1,1,2-trichloro-1,2,2-trifluoroethane","=""76-13-1""",CFC-113,CCl2FCClF2,93.000,0.301000,1.670000e-10,6860.000,5.830000e-10,6520.000,8.900000e-10,2830.000,2.960000e-12,6910.000,2.060000e-12,5210.000,NaN,NaN
9,"1,1,1-trichloro-2,2,2-trifluoroethane","=""354-58-5""",CFC-113a,CCl3CF3,55.000,0.241000,1.240000e-10,5110.000,3.510000e-10,3930.000,4.220000e-10,1350.000,1.730000e-12,4030.000,9.170000e-13,2320.000,NaN,NaN


In [14]:
# hand-edit what's left
df.loc[87, 'Name'] = '1,3,3,4,4-pentafluorocyclobutene'
df.loc[160, 'Name'] = '1,1,1,2,2-pentafluoro-2-(1,1,2,2-tetrafluoroethoxy)ethane'
df.loc[165, 'Name'] = '1-(2,2-difluoroethoxy)-1,1,2,2,2-pentafluoroethane'
df.loc[170, 'Name'] = '1-(2,2-difluoroethoxy)-1,1,2,2-tetrafluoroethane'
df.loc[181, 'Name'] = '2-(difluoromethoxymethyl)-1,1,1,2,3,3,3-heptafluoropropane'
df.loc[190, 'Name'] = '1,1,3,3,4,4,6,6,7,7,9,9,10,10,12,12-hexadecafluoro-2,5,8,11-tetraoxadodecane'
df.loc[192, 'Name'] = '2-ethoxy-3,3,4,4,5-pentafluorotetrahydro-2,5-bis[1,2,2,2-tetrafluoro-1-(trifluoromethyl)ethyl]-furan'
df.loc[195, 'Name'] = '1,1,2,2-Tetrafluoro-1-methoxyethane'
df.loc[202, 'Name'] = '1‐(difluoro(trifluoromethoxy)methoxy)‐1,1,2,3,3,3‐hexafluoro‐2‐(trifluoromethoxy)propane'
df.loc[223, 'Name'] = ''

In [15]:
df.to_csv('../data_output/7sm/metrics_supplement_cleaned.csv', index=False)